2024 May 30, updated  
for CrySPY 1.3.0

# Import and setting

In [ ]:
# ---------- import
import gzip
import pickle

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.animation import FuncAnimation
import numpy as np

In [ ]:
rcParams_dict = {
    # ---------- figure
    'figure.figsize': [8, 6],
    'figure.dpi': 120,
    'figure.facecolor': 'white',
    # ---------- axes
    'axes.grid': True,
    'axes.linewidth': 1.5,
    # ---------- ticks
    'xtick.direction': 'in',
    'ytick.direction': 'in',
    'xtick.major.width': 1.0,
    'ytick.major.width': 1.0,
    'xtick.major.size': 8.0,
    'ytick.major.size': 8.0,
    # ---------- lines
    'lines.linewidth': 2.5,
    'lines.markersize': 12,
    # ---------- grid
    'grid.linestyle': ':',
    # ---------- font
    'font.family': 'Times New Roman',
    'mathtext.fontset': 'cm',
    #'mathtext.fontset': 'stix',
    'font.size': 20,
    'axes.labelsize': 26,
    'legend.fontsize': 26,
    'svg.fonttype': 'path',  # Embed characters as paths
    #'svg.fonttype': 'none',  # Assume fonts are installed on the machine
    'pdf.fonttype': 42,  # embed fonts in PDF using type42 (True type)
}

plt.rcParams.update(rcParams_dict)

# Data

In [ ]:
def load_data(filename):
    if filename.endswith('.gz'):
        with gzip.open(filename, 'rb') as f:
            return pickle.load(f)
    else:
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [ ]:
rslt_data = load_data('./pkl_data/rslt_data.pkl')

# ---------- sort Selection
#rslt_data.head(10)

# ---------- sort by Energy
rslt_data.sort_values(by=['E_eV_atom']).head(10)

In [ ]:
# ---------- Number of structures
ndata = len(rslt_data)
print(f'Number of data (finished): {ndata}')

# ---------- check success and error
nsuccess = rslt_data['E_eV_atom'].count()
nerror = ndata - nsuccess
print(f'Success: {nsuccess}')
print(f'Error: {nerror}')

# ---------- minimum
Emin = rslt_data['E_eV_atom'].min()
print(f'Emin: {Emin} eV/atom')

In [ ]:
id_select_hist = load_data('pkl_data/id_select_hist.pkl')
tot_step_select = load_data('pkl_data/tot_step_select.pkl')
laqa_step = load_data('pkl_data/laqa_step.pkl')
laqa_energy = load_data('pkl_data/laqa_energy.pkl')

# Energy vs. ID

In [ ]:
title = 'LAQA for Si$_{8}$'
dx = 1
xmin = -dx
xmax = max(rslt_data.index) + dx
ymin = -0.2
ymax = 3.0

In [ ]:
fig, ax = plt.subplots()

# ---------- axis
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])

# ---------- hline at zero
ax.hlines(0.0, xmin, xmax, 'k', '--')

# ---------- plot
ax.plot(rslt_data.index, rslt_data['E_eV_atom'] - Emin, 'o', ms=15, mew=2.0, alpha=0.8)

# ---------- title and label
ax.set_title(title)
ax.set_xlabel('Structure ID')
ax.set_ylabel('Energy (eV/atom)')

In [ ]:
filename = 'Si8_LAQA.png'
#filename = 'Si8_LAQA.pdf'

In [ ]:
# ---------- save figure
fig.savefig(filename, bbox_inches='tight')    # PNG, PDF
#fig.savefig(filename, bbox_inches='tight', dpi=300)    # high dpi PNG

## LAQA process

In [ ]:
# ---------- top 10
rslt_data.sort_values(by=['E_eV_atom']).head(10)

In [ ]:
# select the structure IDs you want to stand out by changing the color in the graph
stable_IDs = [2]    # manual input

## Required optimization steps

In [ ]:
id_done = rslt_data.index.values

In [ ]:
req_step = {}    # number of steps for each structure
req_step_done = 0   # total steps for completed structures
for cid, steps in laqa_step.items():
        req_step[cid] = sum(steps)
        if cid in id_done:
            req_step_done += sum(steps)

In [ ]:
print('Total optimization steps:', sum(tot_step_select))
print('Number of completed structures:', len(id_done))    # include skip
print('Total optimization steps for completed structures:', req_step_done)
print('Average number of optimization steps for completed structures:', req_step_done/len(id_done))

## Energy vs. step for png figure

In [ ]:
Emin

In [ ]:
title = 'LAQA for Si$_{8}$'
dx = 2
sps = 50    # step per selection
xmin = 0
xmax = max(req_step.values())+dx
ymin = -0.2
ymax = 20

In [ ]:
fig, ax = plt.subplots()

# ---------- axis
# ------ x axis
ax.set_xlim([xmin, xmax])
major_x = MultipleLocator(sps)
minor_x = MultipleLocator(sps/2)
ax.xaxis.set_major_locator(major_x)
ax.xaxis.set_minor_locator(minor_x)
# ------ y axis
ax.set_ylim([ymin, ymax])

# ---------- gird
ax.grid(which='minor')    # grid: major --> minor

# ---------- title and label
ax.set_title(title)
ax.set_xlabel('Number of step')
ax.set_ylabel('Energy (eV/atom)')
#plt.tight_layout()

# ---------- hline at zero
ax.hlines(0.0, xmin, xmax, 'k', '--')

# ---------- plot
for cid in laqa_energy.keys():
    if cid not in stable_IDs:
        ax.plot(np.cumsum(laqa_step[cid]), laqa_energy[cid] - Emin, color='royalblue', linewidth=1.5)
        if cid in id_done:
            ax.plot(req_step[cid], rslt_data.loc[cid, 'E_eV_atom'] - Emin, 'o', color='royalblue', ms=6, mew=2.0, alpha=0.8)
    else:
        ax.plot(np.cumsum(laqa_step[cid]), laqa_energy[cid] - Emin, color='red')
        if cid in id_done:
            ax.plot(req_step[cid], rslt_data.loc[cid, 'E_eV_atom'] - Emin, 'o', color='red', ms=6, mew=2.0, alpha=0.8)

In [ ]:
filename = 'Si8_LAQA_step.png'
#filename = 'Si8_LAQA_step.pdf'

In [ ]:
# ---------- save figure
fig.savefig(filename, bbox_inches='tight')    # PNG, PDF
#fig.savefig(filename, bbox_inches='tight', dpi=300)    # high dpi PNG

## Energy gif anime

In [ ]:
filename = 'Si8_LAQA_step.gif'
# use same values as above
# title = 'LAQA for Si$_{8}$'
# dx = 2
# sps = 50    # step per selection
# xmin = 0
# xmax = max(req_step.values())+dx
# ymin = -0.2
# ymax = 20

In [ ]:
# ---------- figure
fig2, ax2 = plt.subplots()

# --------- initialize
lines = []
xdata = []
ydata = []
num_select = {}
for cid in range(len(laqa_energy)):
    xdata.append([laqa_step[cid][0]])
    ydata.append([laqa_energy[cid][0] - Emin])
    num_select[cid] = 0
    if cid in stable_IDs:
        lines.append(ax2.plot([], [], color='red')[0])
    else:
        lines.append(ax2.plot([], [], color='royalblue', linewidth=1.5)[0])

ax2.set_xlim([xmin, xmax])
major_x = MultipleLocator(sps)
minor_x = MultipleLocator(sps/2)
ax2.xaxis.set_major_locator(major_x)
ax2.xaxis.set_minor_locator(minor_x)
ax2.grid(which='minor')    # grid: major --> minor
ax2.set_ylim([ymin, ymax])
ax2.set_title(title)
ax2.set_xlabel('Number of step')
ax2.set_ylabel('Energy (eV/atom)')
ax2.hlines(0.0, xmin, xmax, 'k', '--')


def init():
    return lines

# ---------- animate function
# frame i --> 0, 1, 2, ...
def animate(i):
    for cid in id_select_hist[i]:
        num_select[cid] += 1
        try:
            xdata[cid].append(laqa_step[cid][num_select[cid]])
            ydata[cid].append(laqa_energy[cid][num_select[cid]] - Emin)
            lines[cid].set_data(np.cumsum(xdata[cid]), ydata[cid])
        except IndexError:
            # no latest results in the middle of calculations
            break
    return lines


# ---------- call the animator
anim = FuncAnimation(fig2, animate, init_func=init, frames=len(id_select_hist), blit=True)
plt.close()    # not to show the figure in jupyter, only animation

# ---------- show or save: chose only one, or you face an error
#HTML(anim.to_jshtml())    # show the animation in jupyter
anim.save(filename, writer='pillow')    # default: fps=5
#anim.save('LAQA.gif', writer='pillow', fps=20)